# HW 2

Muyuan Zhang

u1430770

06/02/2023

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.stats.proportion import proportions_ztest

## Part 1

Null Hypothesis: Logo B is not better than Logo A.

In [ ]:
successes = np.array([500, 550])
stat, p_value = proportions_ztest(count=successes, nobs=1000)

print('z_value: %0.3f, p_value: %0.3f' % (stat, p_value))

if p_value > 0.05:
    print("Logo B is not better than Logo A at 5% significance level.")
else:
    print("Logo B is better than Logo A at 5% significance level.")

if p_value > 0.01:
    print("Logo B is not better than Logo A at 1% significance level.")
else:
    print("Logo B is better than Logo A at 1% significance level.")


Conclusion: Logo B is better than Logo A at 5% significance level, but no better than Logo A at 1% significance level.

If b is clicked 560 or 570 times, `p value` will be smaller than 0.025.

## Part 2

### Task 1

In [ ]:
df_1 = pd.read_csv("realEstate1.csv")
df_2 = pd.read_csv("realEstate2.csv")
df = pd.concat([df_1, df_2])
df

### Task 2

In [ ]:
# Only keep houses with List Price between 200,000 and 1,000,000 dollars
df = df[(df.LstPrice >= 200000) & (df.LstPrice <= 1000000)]
df.reset_index(inplace=True, drop=True)

# Remove columns that don't contribute to the value of the house
df = df[['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'PropType', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']]

# Convert 'TotSqf' to an integer.
df.TotSqf = pd.Series(df.TotSqf).str.replace(',', '', regex=True)
df.TotSqf = pd.to_numeric(df.TotSqf)

Prop_Type_SingleFamily_Data = []

for i in df['PropType']:
    if i == 'Single Family':
        Prop_Type_SingleFamily_Data.append(1)
    else:
        Prop_Type_SingleFamily_Data.append(0)

df['Prop_Type_SingleFamily'] = Prop_Type_SingleFamily_Data

# Remove the listings with erroneous 'Longitude' (one has Longitude = 0)
# and 'Taxes' values (two have unreasonably large values)
df = df[(df.Longitude != 0) & (df.Taxes < 10000)]
df.reset_index(inplace=True, drop=True)
df

### Task 3

In [ ]:
df.describe()

The dataset has 260 items and 15 variables. The minimum value of `Taxes` is 1, which is sort of weird.

In [ ]:
# Make a bar chart showing the breakdown of the different types of houses
df['PropType'].hist()

In [ ]:
corr_df = df[['Acres', 'Deck', 'GaragCap', 'Latitude', 'Longitude', 'LstPrice', 'Patio', 'PkgSpacs', 'Prop_Type_SingleFamily', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']]

corrMatrix = corr_df.corr()
plt.figure(figsize=(14,14))
sns.heatmap(corrMatrix, annot=True, vmin=-1, vmax=1)

In [ ]:

pd.plotting.scatter_matrix(df[['Acres', 'LstPrice', 'SoldPrice', 'Taxes', 'TotBed', 'TotBth', 'TotSqf', 'YearBlt']], figsize=[14,14])

plt.show()

`SoldPrice` & `LstPrice` are strongly correlated.

With larger `TotSqf`s, the properties are more likely to have higher `SoldPrice`, `LstPrice`, `Taxes`.

### Task 4

In [ ]:
geoplot = df.plot(x='Longitude', y='Latitude', c='LstPrice', colormap='viridis', figsize=[10,10], kind='scatter')

The prices of properties go higher on the northern or eastern side of the city.

### Task 5

In [ ]:
result = smf.ols(formula='SoldPrice ~ TotBed', data=df).fit()
print(result.summary())

In [ ]:
result = smf.ols(formula='SoldPrice ~ TotSqf', data=df).fit()
print(result.summary())

In [ ]:
result = smf.ols(formula='SoldPrice ~ Taxes', data=df).fit()
print(result.summary())

In [ ]:
result = smf.ols(formula='SoldPrice ~ LstPrice', data=df).fit()
print(result.summary())

The R-squared value for `SoldPrice` ~ `LstPrice` model is 0.990, which indicates that the model predicts 99% of the relationship.

beta_1 for this model is 0.9441, which means that the regression coefficient for `SoldPrice` ~ `LstPrice` model is 0.9441 and `SoldPrice` will be slightly less than `LstPrice`.

In [ ]:
# Make a plot of list price vs. sold price and overlay the prediction coming from your regression model
sns.regplot(x='LstPrice', y='SoldPrice', data=df)

### Task 6

In [ ]:
result = smf.ols(formula='SoldPrice ~ Acres + GaragCap + Latitude + Longitude + TotBed + TotBth + TotSqf', data=df).fit()
print(result.summary())

* Which variables are the best predictors for the Sold Price?

`Acres`, `GaragCap`, `Longitude`, and `TotSqf` are the best predictors for `SoldPrice` due to small p values.

* Often the price per square foot for a house is advertised. Is this what the coefficient for TotSqf is measuring? Provide an interpretation for the coefficient for TotSqf.

Yes. The coefficient for `TotSqf` means that beta_1 for this model is 90.8848 which is the price per square foot for a house.

* Estimate the value that each Garage space adds to a house.

4.165e+04 dollars.

* Does latitude or longitude have an impact on house price? Explain.

`Latitude` does not have a significant impact on house price. `Longitude` seems to have, but is only applicable within a given range, which is inside the city.

* If we wanted to start a 'house flipping' company, we'd have to be able to do a better job of predicting the sold price than the list price does. How does your model compare?

Remove the fields that do not have an impact on `SoldPrice` and build the model again.

In [ ]:
result = smf.ols(formula='SoldPrice ~ Acres + GaragCap + Longitude + TotSqf', data=df).fit()
print(result.summary())

The R-squared value for our model is 0.803, which is lower than that for `SoldPrice` ~ `LstPrice` (0.990). Therefore our model cannot do a better job of predicting the sold price than the list price does.

### Task 7

In [ ]:
color_dict = {'Single Family', 'Condo', 'Townhouse'}
PropType = df.PropType.astype('category').cat.codes
plt.figure(figsize=(10,10))
scatter = plt.scatter(y=df.TotSqf, x=df.SoldPrice, c=PropType, s=10)
plt.xlabel('Sold Price')
plt.ylabel('Total Sq. Ft.')
prop_names = ['Condo', 'Single Family', 'Townhouse']
plt.legend(handles=scatter.legend_elements()[0],
           labels=prop_names,
           title="Property Type")

Property type is no longer predictable because `TotSqf` is much more related to `SoldPrice` than `PropType`.